# Navier Stokes Equation Reduced Basis Method 
Incompressible timedependet Navier Stokes equation

Find $(u,p):[0,T] \to (H_{0,D}^1)^d \times L^2$, s.t.

\begin{align}
\int_{\Omega} \partial_t u \cdot v + \int_{\Omega} \nu \nabla u \nabla v + u \cdot \nabla u v - \int_{\Omega} \operatorname{div}(v) p &= \int f v  && \forall v \in (H_{0,D}^1)^d, \\ 
- \int_{\Omega} \operatorname{div}(u) q &= 0 && \forall q \in L^2, \\
\quad u(t=0) & = u_0
\end{align}

In [ ]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np
from math import pi

In [ ]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(3)
Draw(mesh)
# viscosity
nu = 0.001
dt = 0.001
dn = 1./dt
z = 20  # Number of snapshots

In [ ]:
k = 3
V = VectorH1(mesh,order=k, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q])

In [ ]:
def ChebPoints(a,b,n):
    x_i = []
    for i in range(0,n+1):
        fac = (2*i+1)/(2*n+2)
        x_ch = np.cos(pi*fac)
        x_i.append((a+b)/2+((b-a)/2)*x_ch)
    return np.sort(x_i)

In [ ]:
gfu_N = GridFunction(X)
gfu = GridFunction(X)
velocity = gfu.components[0]
velocity_N = gfu_N.components[0]


uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

Draw(gfu.components[0],mesh,"gfu",sd=3)

In [ ]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res
Draw(gfu.components[0],mesh,"gfu",sd=3)

In [ ]:
# matrix for implicit part of IMEX(1) scheme:
mstar = BilinearForm(X)

mstar += InnerProduct(u,v)*dx + dt*stokes

mstar.Assemble()

mh = BilinearForm(X)

mh += InnerProduct(u,v)*dx
mh.Assemble()

inv = mstar.mat.Inverse(X.FreeDofs())


conv = LinearForm(X)
conv += InnerProduct(grad(velocity)*velocity,v)*dx

In [ ]:
rows,cols,vals = a.mat.COO()
Ah = sp.csr_matrix((vals,(rows,cols)))
Fh = f.vec
rows,cols,vals = mh.mat.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))

In [ ]:
t = 0

erg_h = np.zeros([len(gfu.vec),int(dn)+1])

V = np.zeros([len(gfu.vec),z]) # creat the orthogonal transformationsmatrix for the reducebasis method
line = np.linspace(0,dn-1,z) # stors the number of linear distributed number of iteration from the solution
S_erg = np.zeros([len(gfu.vec),z])

In [ ]:
# implicit Euler/explicit Euler splitting method:
V_sb = MultiVector(gfu.vec, 1)
V_sb[0] = gfu.vec

V[:,0] = np.array(V_sb[0])/np.linalg.norm(np.array(V_sb[0]))
erg_h[:,0] = gfu.vec
S_erg[:,0] = gfu.vec

o = 1
e = int(line[o])
for i in range(int(dn)):
    conv.Assemble()
    res.data = a.mat * gfu.vec + conv.vec
    gfu.vec.data -= dt * inv * res
    
    for r in line:
        if int(r) == i: #desided which solution will be orthogonalirsed
            if i > 0:
                with TaskManager():
                    V_sb.AppendOrthogonalize(gfu.vec)
                V[:,o] = np.array(V_sb[o])
                S_erg[:,o] = gfu.vec
                o = o + 1
    erg_h[:,i+1] = gfu.vec #stores the solution for comparing later
    t = t + dt
    Redraw()

## POD method
A other way to get the transforamtions matrix is the POD method.

In [ ]:
def POD(S):
    V = np.zeros([len(gfu.vec),z])
    
    #splitting the snapshotsolution like in the paper
    for i in range(S.shape[1]):
        S[:,i] = S[:,i] - 1/S.shape[0] * np.sum(S[:,i])
    
    if S.shape[1] <= S.shape[0]:
        C = np.transpose(S).dot(S)
        U, Sig, VH = np.linalg.svd(C, full_matrices=True)
        VH = np.transpose(VH)
        for i in range(C.shape[1]):
            V[:,i] = 1/np.sqrt(Sig[i]) * S.dot(VH[:,i])
        
        
    return V

In [ ]:
V = POD(S_erg)

In [ ]:
MN = np.transpose(V).dot(Mh.dot(V))
AN = np.transpose(V).dot(Ah.dot(V))

In [ ]:
uN = erg_h[:,0].dot(V)
gfu_N.vec.FV().NumPy()[:] = erg_h[:,0]
gfu.vec.FV().NumPy()[:] = erg_h[:,0]
Draw(gfu_N.components[0],mesh,"gfu_N",sd=3)
Draw(gfu.components[0],mesh,"gfu",sd=3)
Draw(gfu_N.components[0]-gfu.components[0],mesh,"error",sd=3)
err = res.CreateVector()
test = res.CreateVector()

In [ ]:
C0 = LinearForm(X)
C0 += InnerProduct(grad(velocity_N)*velocity_N,v)*dx

In [ ]:
Residium = []
time = []
erg_N = np.zeros([len(gfu_N.vec),int(dn)+1])
erg_N[:,0] = gfu_N.vec

MN_star = MN + dt*AN
inv_N = np.linalg.inv(MN_star)
for i in range(int(dn)):
    err.vec = sqrt (Integrate ((gfu.components[0]-gfu_N.components[0])*(gfu.components[0]-gfu_N.components[0]),mesh))
    Residium.append(err.vec)
    C0.Assemble()
    C0_N = np.transpose(V).dot(C0.vec)
    
    res_N = AN.dot(uN) + C0_N
    uN -= uN + dt * inv_N.dot(res_N)
    gfu.vec.FV().NumPy()[:] = erg_h[:,i+1]
    velocity_N = gfu_N.components[0]
    gfu_N.vec.FV().NumPy()[:] = (uN).dot(np.transpose(V))
    erg_N[:,i+1] = gfu_N.vec
    time.append(dt*i)
    velocity_N = gfu_N.components[0]
    
   
    Redraw()